In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
/kaggle/input/2021-ai-quiz1-p3/train.csv
/kaggle/input/2021-ai-quiz1-p3/test.csv
/kaggle/input/2021-ai-quiz1-p3/submit_sample.csv
# torch 관련 library 호출

import torch
import torch.optim as optim
import torch.nn as nn

# GPU 사용을 위한 device 선언

device = "cuda" if torch.cuda.is_available() else "cpu"
device
'cpu'
# 데이터 로드

train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p3/submit_sample.csv")
train
Unnamed: 0	0	1	2	3	4	5	6	7	8	...	141	142	143	144	145	146	147	148	149	0.1
0	0	-2.075606	-1.045790	2.126936	0.036825	-0.757574	-0.517365	0.855506	1.051939	0.457736	...	0.766970	-0.424478	-0.124687	-1.496749	0.447682	0.436117	0.456781	-0.871528	2.808375	3
1	1	1.321112	0.592836	0.534154	0.122660	1.182957	-0.673364	-0.182102	1.064393	0.870060	...	0.374759	-0.317582	-0.199934	-0.573055	0.597456	-0.123165	1.199251	-0.920927	1.424777	1
2	2	-0.761193	-0.019730	-0.239907	0.499094	1.304381	-0.561011	0.069747	1.620910	0.118996	...	-1.785485	0.708388	0.001030	-0.015270	1.503353	0.867290	1.289758	-0.995063	-0.750737	2
3	3	-0.117408	0.116545	-0.009745	2.104061	-0.549831	0.623429	0.885584	0.496275	-0.346002	...	-1.107025	-0.314110	1.143012	1.028699	0.271098	0.230415	0.643734	-2.632324	-0.415003	6
4	4	-0.396370	0.426845	-0.250984	-0.651813	1.795055	0.917808	0.349123	-1.391063	1.175008	...	0.595545	-0.752304	-0.414062	0.662435	0.014245	0.087689	0.695441	0.475167	-0.314527	4
...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...
961	961	-0.087658	0.502053	-0.153144	-1.402996	-0.629632	-0.907732	0.724808	-1.723367	-1.083608	...	-0.089831	0.983060	0.218364	0.235335	-1.479324	-2.567603	-0.321805	-1.250793	-2.288816	1
962	962	0.481050	-0.244974	0.390068	-2.094188	-1.778837	-1.525156	-0.293080	-0.374690	1.558995	...	0.287247	0.113489	-1.096534	1.646904	1.081852	0.553180	1.039318	1.006797	1.336859	6
963	963	-0.760314	0.040259	-0.212453	-1.249947	0.036339	1.906293	-0.274605	-0.407281	-1.245358	...	1.460066	0.468226	0.619295	0.146351	-0.362881	0.741775	-0.072357	-0.197916	-0.548080	3
964	964	-0.267422	0.838640	-0.222198	-0.952942	0.468293	0.900771	-0.627356	0.517218	1.974085	...	2.055150	1.007875	-0.011870	0.352795	-0.213072	1.968517	-1.882339	1.767359	-1.468716	2
965	965	-1.319565	-1.126696	0.156169	-1.307257	-0.317330	1.714553	0.443122	0.402669	1.907508	...	-1.172470	-1.295797	-2.220556	0.596601	-2.465747	1.625675	1.070389	-0.375185	1.220137	1
966 rows × 152 columns

# 학습 데이터셋 구축

x = train.drop(["Unnamed: 0", "0.1"], axis=1)
y = train["0.1"]
x_test = test.drop(["Unnamed: 0"], axis=1)
# 크게 튀는 값은 없어 보이나, 데이터 자체의 안정화를 위해 가장 기본이 되는 전처리 사용

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)
x_test = sc.transform(x_test)
# 학습 데이터를 tensor화 시킨 후 GPU 연동

x = torch.FloatTensor(np.array(x)).to(device)
y = torch.LongTensor(np.array(y)).to(device)
x_test = torch.FloatTensor(np.array(x_test)).to(device)
# 학습 데이터 형태 출력

print(x.shape)
print(y.shape)
print(x_test.shape)
torch.Size([966, 150])
torch.Size([966])
torch.Size([322, 150])
y.unique()
tensor([0, 1, 2, 3, 4, 5, 6])
# 랜덤 값 고정

torch.manual_seed(1)

# 총 3개의 레이어 사용

linear1 = nn.Linear(150, 450, bias = True)
linear2 = nn.Linear(450, 300, bias = True)
linear3 = nn.Linear(40, 20, bias = True)
linear4 = nn.Linear(300, 7, bias = True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()

# 모델 구성

model = nn.Sequential(linear1, relu, linear2, sigmoid, linear4).to(device)

# optimizer, loss

optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss = nn.CrossEntropyLoss()
# iteration

for stop in range(1500):
    optimizer.zero_grad()
    hypothesis = model(x)
    cost = loss(hypothesis, y)
    cost.backward()
    optimizer.step()
    
    if stop % 100 == 0:
        print(stop, cost.item())
0 2.101914644241333
100 0.017238745465874672
200 0.0048625594936311245
300 0.002484563272446394
400 0.00154509034473449
500 0.0010670615592971444
600 0.0007868750253692269
700 0.0006069521186873317
800 0.00048376075574196875
900 0.0003952813567593694
1000 0.00032935081981122494
1100 0.00027877147658728063
1200 0.00023902812972664833
1300 0.00020717622828669846
1400 0.0001812183327274397
# 결과 확인

hypothesis = model(x_test)
pred = hypothesis.argmax(dim=1).cpu()
result = pred.detach().numpy()
submit["Category"] = result
submit
id	Category
0	0	3
1	1	3
2	2	6
3	3	3
4	4	3
...	...	...
317	317	1
318	318	3
319	319	1
320	320	3
321	321	2
322 rows × 2 columns

submit.to_csv("q3submit.csv", index=False)
P3 코드 설명
Preprocessing
데이터 자체가 이미 PCA를 통해 가공이 된 상태라 처음에는 전처리 없이 진행해보았습니다. StandardScaler를 사용했을 때는 성능에 변화가 없었습니다.

Training
캐글 평가기준이 Categorization Accuracy 였고, 분류할 클래스가 총 7개였기 때문에 Loss Function은 CrossEntropyLoss를 사용하였습니다. 처음에는 레이어를 4개로 구성하였으나, 3개로 줄였을 때 학습이 더 잘 되었습니다. 반복의 경우, 1000으로 두고 했을 때 최고점이 나왔기 때문에 조금 더 늘린 1500으로 시도해도 성능은 같았습니다.